In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

from qubex.experiment import Experiment

ex = Experiment(
    chip_id="16Q",
    qubits=[8, 10],
    config_dir="/home/shared/config",
)

In [ ]:
ex.tool.print_target_frequencies()
# ex.configure()

In [ ]:
result_rabi = ex.obtain_rabi_params()
result_hpi = ex.calibrate_hpi_pulse()
result_pi = ex.calibrate_pi_pulse()
result_drag_hpi = ex.calibrate_drag_hpi_pulse()
result_drag_pi = ex.calibrate_drag_pi_pulse()
result_classifier = ex.build_classifier()

In [5]:
cr_control = ex.qubit_labels[0]
cr_target = ex.qubit_labels[1]
cr_pair = (cr_control, cr_target)

cr_duration = 100
cr_ramptime = 40

In [ ]:
result_cr = ex.obtain_cr_params(*cr_pair)

In [ ]:
result_cr_calib = ex.calibrate_zx90_by_amplitude(
    *cr_pair,
    duration=cr_duration,
    ramptime=cr_ramptime,
)

root = result_cr_calib["root"]

In [ ]:
result_cr_calib = ex.calibrate_zx90_by_amplitude(
    *cr_pair,
    duration=cr_duration,
    ramptime=cr_ramptime,
    amplitude_range=np.linspace(root - 0.1, root + 0.1, 100),
)

In [ ]:
zx90 = ex.zx90(*cr_pair)

zx90.plot()

In [ ]:
result = ex.repeat_sequence(zx90, repetitions=4)

In [ ]:
result = ex.pulse_tomography(zx90.repeated(4), initial_state={cr_control: "0"})

In [ ]:
result = ex.pulse_tomography(zx90.repeated(4), initial_state={cr_control: "1"})

In [ ]:
ex.measure_bell_state(*cr_pair)

In [ ]:
result_rb = ex.randomized_benchmarking(
    target=f"{cr_control}-{cr_target}",
    n_cliffords_range=np.arange(0, 21, 2),
    n_trials=30,
)

In [ ]:
result_irb = ex.interleaved_randomized_benchmarking(
    target=f"{cr_control}-{cr_target}",
    n_cliffords_range=np.arange(0, 21, 2),
    n_trials=30,
    interleaved_waveform=zx90,
    interleaved_clifford=ex.clifford["ZX90"],
)